In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
import math

In [30]:
#file = r'C:\DTU\PupilProcessing_MATLABscripts\mDBSCAN\dummydata.xlsx'
file = r'C:\DTU\Data\201901_JanuaryExpt\eyeData.xlsx'
df = pd.read_excel(file, header=None)

In [31]:
    
gazePoint = np.array([(float(df[0][i]), float(df[1][i]), i/90) for i in range(len(df[0])) if not np.isnan(df[0][i]) ])
gazePointInd = np.array([i for i in range(len(df[0])) if not np.isnan(df[0][i])])




In [32]:
min_fix = 0.100
min_cluster_size = 2.5
screenx = 1280
screeny = 1080
frame_res = 90.0

# Derived value(s)
min_fix_pts = round(min_fix * frame_res)



In [33]:
def spat_dist(p1, p2):
    return math.hypot(p1[0]-p2[0], p1[1]-p2[1])

In [34]:
def cluster_frames_DBSCAN(npeyeframes):
    '''
    Cluster frames with a derivate of DBSCAN taking into account the time
    
    This DBSCAN variation uses DBSCAN in the local spatial neighbourhood with timing constraints:
    Single or a few outliers followed by additional connected points are considered noise, 
    and more points can be added to the current cluster
    However if enough disjoint points are seen that could make up a cluster (temporally!) it ends the current cluster
    '''

    def query_region(frames, first, reference):
        '''returns absolute indices of frames in neighbourhood of reference, from first to break/len(frames)'''
        i = first
        region = set()
        seq_outside = 0
        while i < len(frames):
            if spat_dist(frames[reference], frames[i]) < min_cluster_size:
                region.add(i)
                seq_outside = 0
            else:
                if i > reference:
                    seq_outside += 1
                    if seq_outside >= min_fix_pts:
                        break
            i += 1
        return region


    # first create labels; 0 means unassigned (so far)
    labels = np.zeros(len(npeyeframes), dtype=int)

    label = 1
    unassigned = 0
    nextp = unassigned
    while nextp < len(labels):
        print ("DBSCAN index %d" % nextp)
        if labels[nextp] == 0:
            neighbours = query_region(npeyeframes, unassigned, nextp)
            if len(neighbours) < min_fix_pts:
                print ("   Too few neighbours; leaving point alone: %d" % len(neighbours))
            else:
                print ("   New cluster: %d [Initial: %d]" % (label, len(neighbours)))
                unvisited = neighbours
                unvisited.remove(nextp)
                visited = set()
                visited.add(nextp)

                while len(unvisited) > 0:
                    first = unvisited.pop()

                    visited.add(first)
                    neighbours = query_region(npeyeframes, unassigned, first)
                    if len(neighbours) >= min_fix_pts:  # SHOULD THIS BE > 0 only?
                        unvisited.update(neighbours - visited)

                for v in visited:
                    assert labels[v] == 0, "Reassigning point to new cluster, should not take place"
                    labels[v] = label
                    nextp = max(nextp, v)

                unassigned = nextp + 1
                label += 1
                print ("   Next unassigned: %d [Final: %d]" % (unassigned, len(visited)))
        else:
            print ("   Already assigned to %d" % labels[nextp])

        nextp += 1

    # remove any empty last cluster
    if len(npeyeframes[labels==label])==0:
        print ("   Last cluster (%d) is empty, removing" % (label))
        label -= 1

    # calculate the cluster centers...
    print ("   Ended up with %d clusters" % label)
    cluster_centers = np.empty([label+1, 3*4])
    for l in range(label+1):
        # print ("   Fixing frames in  clusters %d" % l)

        myframes = npeyeframes[labels==l]
        mylen = len(myframes)
        if (mylen > 0):
            
            cmean = np.mean(myframes, 0)
            cmin = np.amin(myframes, 0)
            cmax = np.amax(myframes, 0)
            cstd = np.std(myframes, 0)
            cluster_centers[l] = np.concatenate((cmean, cmin, cmax, cstd))

    return (labels, cluster_centers)

In [35]:
def cluster_frames(npeyeframes):
    '''
    Cluster the npeyeframes that each have (x,y,t) positions; need not be equitemporally sampled
    
    Returns a tupple of labels and centers
    where len(labels)==len(npeyeframes), and labels[i] is index into clusters for frame i
    and centers contains the (x,y,t) coordinates of each cluster found
    '''

    return cluster_frames_DBSCAN(npeyeframes)

In [36]:
# Cluster the data into and centers and labels
labels, cluster_centers = cluster_frames(gazePoint)

DBSCAN index 0
   New cluster: 1 [Initial: 26]
   Next unassigned: 61 [Final: 61]
DBSCAN index 61
   Too few neighbours; leaving point alone: 1
DBSCAN index 62
   Too few neighbours; leaving point alone: 2
DBSCAN index 63
   New cluster: 2 [Initial: 26]
   Next unassigned: 141 [Final: 79]
DBSCAN index 141
   Too few neighbours; leaving point alone: 1
DBSCAN index 142
   Too few neighbours; leaving point alone: 1
DBSCAN index 143
   Too few neighbours; leaving point alone: 1
DBSCAN index 144
   Too few neighbours; leaving point alone: 1
DBSCAN index 145
   New cluster: 3 [Initial: 14]
   Next unassigned: 185 [Final: 40]
DBSCAN index 185
   Too few neighbours; leaving point alone: 7
DBSCAN index 186
   New cluster: 4 [Initial: 11]
   Next unassigned: 196 [Final: 11]
DBSCAN index 196
   Too few neighbours; leaving point alone: 1
DBSCAN index 197
   Too few neighbours; leaving point alone: 1
DBSCAN index 198
   New cluster: 5 [Initial: 14]
   Next unassigned: 224 [Final: 26]
DBSCAN index 2

   Next unassigned: 3342 [Final: 110]
DBSCAN index 3342
   Too few neighbours; leaving point alone: 1
DBSCAN index 3343
   New cluster: 35 [Initial: 27]
   Next unassigned: 3537 [Final: 194]
DBSCAN index 3537
   Too few neighbours; leaving point alone: 1
DBSCAN index 3538
   New cluster: 36 [Initial: 66]
   Next unassigned: 3607 [Final: 69]
DBSCAN index 3607
   Too few neighbours; leaving point alone: 1
DBSCAN index 3608
   Too few neighbours; leaving point alone: 1
DBSCAN index 3609
   Too few neighbours; leaving point alone: 1
DBSCAN index 3610
   New cluster: 37 [Initial: 9]
   Next unassigned: 3644 [Final: 34]
DBSCAN index 3644
   Too few neighbours; leaving point alone: 5
DBSCAN index 3645
   Too few neighbours; leaving point alone: 6
DBSCAN index 3646
   Too few neighbours; leaving point alone: 6
DBSCAN index 3647
   Too few neighbours; leaving point alone: 6
DBSCAN index 3648
   Too few neighbours; leaving point alone: 6
DBSCAN index 3649
   New cluster: 38 [Initial: 81]
   Next

   Next unassigned: 6589 [Final: 140]
DBSCAN index 6589
   Too few neighbours; leaving point alone: 2
DBSCAN index 6590
   New cluster: 90 [Initial: 29]
   Next unassigned: 6619 [Final: 30]
DBSCAN index 6619
   Too few neighbours; leaving point alone: 1
DBSCAN index 6620
   New cluster: 91 [Initial: 25]
   Next unassigned: 6657 [Final: 37]
DBSCAN index 6657
   Too few neighbours; leaving point alone: 1
DBSCAN index 6658
   Too few neighbours; leaving point alone: 1
DBSCAN index 6659
   Too few neighbours; leaving point alone: 4
DBSCAN index 6660
   Too few neighbours; leaving point alone: 7
DBSCAN index 6661
   New cluster: 92 [Initial: 10]
   Next unassigned: 6748 [Final: 89]
DBSCAN index 6748
   Too few neighbours; leaving point alone: 1
DBSCAN index 6749
   New cluster: 93 [Initial: 57]
   Next unassigned: 6830 [Final: 81]
DBSCAN index 6830
   New cluster: 94 [Initial: 35]
   Next unassigned: 6866 [Final: 36]
DBSCAN index 6866
   New cluster: 95 [Initial: 36]
   Next unassigned: 690

   Next unassigned: 11010 [Final: 227]
DBSCAN index 11010
   Too few neighbours; leaving point alone: 2
DBSCAN index 11011
   Too few neighbours; leaving point alone: 3
DBSCAN index 11012
   New cluster: 135 [Initial: 18]
   Next unassigned: 11115 [Final: 104]
DBSCAN index 11115
   New cluster: 136 [Initial: 86]
   Next unassigned: 11202 [Final: 87]
DBSCAN index 11202
   Too few neighbours; leaving point alone: 1
DBSCAN index 11203
   Too few neighbours; leaving point alone: 2
DBSCAN index 11204
   Too few neighbours; leaving point alone: 2
DBSCAN index 11205
   New cluster: 137 [Initial: 38]
   Next unassigned: 11328 [Final: 124]
DBSCAN index 11328
   Too few neighbours; leaving point alone: 3
DBSCAN index 11329
   New cluster: 138 [Initial: 59]
   Next unassigned: 11412 [Final: 84]
DBSCAN index 11412
   Too few neighbours; leaving point alone: 2
DBSCAN index 11413
   Too few neighbours; leaving point alone: 2
DBSCAN index 11414
   Too few neighbours; leaving point alone: 1
DBSCAN ind

   Next unassigned: 14860 [Final: 112]
DBSCAN index 14860
   Too few neighbours; leaving point alone: 1
DBSCAN index 14861
   Too few neighbours; leaving point alone: 7
DBSCAN index 14862
   Too few neighbours; leaving point alone: 7
DBSCAN index 14863
   Too few neighbours; leaving point alone: 7
DBSCAN index 14864
   Too few neighbours; leaving point alone: 7
DBSCAN index 14865
   Too few neighbours; leaving point alone: 7
DBSCAN index 14866
   Too few neighbours; leaving point alone: 7
DBSCAN index 14867
   Too few neighbours; leaving point alone: 7
DBSCAN index 14868
   Too few neighbours; leaving point alone: 1
DBSCAN index 14869
   Too few neighbours; leaving point alone: 2
DBSCAN index 14870
   Too few neighbours; leaving point alone: 2
DBSCAN index 14871
   Too few neighbours; leaving point alone: 6
DBSCAN index 14872
   Too few neighbours; leaving point alone: 6
DBSCAN index 14873
   Too few neighbours; leaving point alone: 6
DBSCAN index 14874
   Too few neighbours; leaving p

...

   Next unassigned: 93617 [Final: 123]
DBSCAN index 93617
   Too few neighbours; leaving point alone: 1
DBSCAN index 93618
   Too few neighbours; leaving point alone: 1
DBSCAN index 93619
   Too few neighbours; leaving point alone: 1
DBSCAN index 93620
   New cluster: 942 [Initial: 37]
   Next unassigned: 93733 [Final: 112]
DBSCAN index 93733
   Too few neighbours; leaving point alone: 2
DBSCAN index 93734
   Too few neighbours; leaving point alone: 2
DBSCAN index 93735
   Too few neighbours; leaving point alone: 4
DBSCAN index 93736
   Too few neighbours; leaving point alone: 3
DBSCAN index 93737
   New cluster: 943 [Initial: 48]
   Next unassigned: 93783 [Final: 48]
DBSCAN index 93783
   Too few neighbours; leaving point alone: 1
DBSCAN index 93784
   Too few neighbours; leaving point alone: 1
DBSCAN index 93785
   Too few neighbours; leaving point alone: 1
DBSCAN index 93786
   Too few neighbours; leaving point alone: 1
DBSCAN index 93787
   Too few neighbours; leaving point alone: 

   Next unassigned: 97458 [Final: 123]
DBSCAN index 97458
   New cluster: 991 [Initial: 89]
   Next unassigned: 97639 [Final: 180]
DBSCAN index 97639
   Too few neighbours; leaving point alone: 2
DBSCAN index 97640
   Too few neighbours; leaving point alone: 3
DBSCAN index 97641
   Too few neighbours; leaving point alone: 2
DBSCAN index 97642
   Too few neighbours; leaving point alone: 1
DBSCAN index 97643
   Too few neighbours; leaving point alone: 2
DBSCAN index 97644
   Too few neighbours; leaving point alone: 2
DBSCAN index 97645
   New cluster: 992 [Initial: 41]
   Next unassigned: 97723 [Final: 78]
DBSCAN index 97723
   Too few neighbours; leaving point alone: 1
DBSCAN index 97724
   Too few neighbours; leaving point alone: 2
DBSCAN index 97725
   Too few neighbours; leaving point alone: 2
DBSCAN index 97726
   Too few neighbours; leaving point alone: 3
DBSCAN index 97727
   New cluster: 993 [Initial: 35]
   Next unassigned: 97810 [Final: 82]
DBSCAN index 97810
   Too few neighbo

   Next unassigned: 100443 [Final: 181]
DBSCAN index 100443
   Too few neighbours; leaving point alone: 2
DBSCAN index 100444
   Too few neighbours; leaving point alone: 2
DBSCAN index 100445
   Too few neighbours; leaving point alone: 1
DBSCAN index 100446
   Too few neighbours; leaving point alone: 3
DBSCAN index 100447
   New cluster: 1024 [Initial: 71]
   Next unassigned: 100526 [Final: 80]
DBSCAN index 100526
   Too few neighbours; leaving point alone: 3
DBSCAN index 100527
   New cluster: 1025 [Initial: 13]
   Next unassigned: 100641 [Final: 115]
DBSCAN index 100641
   Too few neighbours; leaving point alone: 1
DBSCAN index 100642
   Too few neighbours; leaving point alone: 1
DBSCAN index 100643
   Too few neighbours; leaving point alone: 1
DBSCAN index 100644
   Too few neighbours; leaving point alone: 2
DBSCAN index 100645
   Too few neighbours; leaving point alone: 3
DBSCAN index 100646
   New cluster: 1026 [Initial: 31]
   Next unassigned: 100760 [Final: 115]
DBSCAN index 100

   Next unassigned: 102521 [Final: 83]
DBSCAN index 102521
   Too few neighbours; leaving point alone: 1
DBSCAN index 102522
   Too few neighbours; leaving point alone: 1
DBSCAN index 102523
   Too few neighbours; leaving point alone: 2
DBSCAN index 102524
   Too few neighbours; leaving point alone: 2
DBSCAN index 102525
   Too few neighbours; leaving point alone: 1
DBSCAN index 102526
   New cluster: 1051 [Initial: 10]
   Next unassigned: 102536 [Final: 10]
DBSCAN index 102536
   Too few neighbours; leaving point alone: 3
DBSCAN index 102537
   Too few neighbours; leaving point alone: 3
DBSCAN index 102538
   Too few neighbours; leaving point alone: 4
DBSCAN index 102539
   New cluster: 1052 [Initial: 17]
   Next unassigned: 102565 [Final: 28]
DBSCAN index 102565
   Too few neighbours; leaving point alone: 1
DBSCAN index 102566
   Too few neighbours; leaving point alone: 1
DBSCAN index 102567
   New cluster: 1053 [Initial: 35]
   Next unassigned: 102695 [Final: 128]
DBSCAN index 10269

   Next unassigned: 103820 [Final: 43]
DBSCAN index 103820
   Too few neighbours; leaving point alone: 3
DBSCAN index 103821
   Too few neighbours; leaving point alone: 3
DBSCAN index 103822
   Too few neighbours; leaving point alone: 3
DBSCAN index 103823
   Too few neighbours; leaving point alone: 6
DBSCAN index 103824
   Too few neighbours; leaving point alone: 6
DBSCAN index 103825
   Too few neighbours; leaving point alone: 7
DBSCAN index 103826
   New cluster: 1073 [Initial: 9]
   Next unassigned: 103939 [Final: 116]
DBSCAN index 103939
   Too few neighbours; leaving point alone: 2
DBSCAN index 103940
   Too few neighbours; leaving point alone: 3
DBSCAN index 103941
   Too few neighbours; leaving point alone: 2
DBSCAN index 103942
   Too few neighbours; leaving point alone: 2
DBSCAN index 103943
   Too few neighbours; leaving point alone: 3
DBSCAN index 103944
   Too few neighbours; leaving point alone: 2
DBSCAN index 103945
   Too few neighbours; leaving point alone: 3
DBSCAN in

In [37]:
# combine labels with nan
labels_all = np.array([np.nan]*len(df))
labels_all[gazePointInd] = labels


In [38]:
#plt.figure(1)
radii = [np.sqrt((i[3]-i[6])**2+(i[4]-i[7])**2)/2 for i in cluster_centers]
fig, ax = plt.subplots()
plt.plot(gazePoint[:,0], gazePoint[:,1], '-oc', alpha = 0.075)

for i,c in enumerate(cluster_centers):
    
    circle1 = plt.Circle((c[0], c[1]), 2.5, color='r', fill=False)
    ax.add_artist(circle1)
    ax.set_xlim((-5, 65))
    ax.set_ylim((-5, 80))
    
    


In [39]:
saccadeAmplitude, saccadeSamples = 0,0
saccadeAmplitudeList, saccadeDurationList  = list(), list()


for i,l in enumerate(labels_all[1:-1]):
    
    if not np.isnan(l):# current point is not a nan value
        if l == 0: # current point is not clustered            
            saccadeSamples = saccadeSamples + 1 
                
            if not np.isnan(labels_all[i]): # checking the previous point
                point_current = (df[3][i+1], df[4][i+1])
                point_previous = (df[3][i], df[4][i])
                saccadeAmplitude = saccadeAmplitude + spat_dist(point_current, point_previous)
                
                
            if not np.isnan(labels_all[i+2]): # checking the subsequent point
                
                if labels_all[i+2] > 0:
                    point_current = (df[3][i+1], df[4][i+1])
                    point_next = (df[3][i+2], df[4][i+2])
                
                    saccadeAmplitude = saccadeAmplitude + spat_dist(point_current, point_next)
                    saccadeAmplitudeList.append(saccadeAmplitude)
                    saccadeDurationList.append(saccadeSamples/frame_res)
                    saccadeAmplitude, saccadeSamples = 0, 0
                    
            else:
                # next point is a blink, the fixation should stop    
                saccadeAmplitudeList.append(saccadeAmplitude)
                saccadeDurationList.append((saccadeSamples/frame_res))
                saccadeAmplitude, saccadeSamples = 0, 0
                
                
                

In [43]:
plt.figure()
plt.plot(saccadeDurationList, saccadeAmplitudeList, 'o')
plt.ylabel('Saccade amplitude [in degrees]')
plt.xlabel('Saccade duration [in s]')

Text(0.5, 0, 'Saccade duration [in s]')

In [44]:
saccadeVelocityList = np.array(saccadeAmplitudeList)/(np.array(saccadeDurationList))

In [45]:
plt.figure()
plt.plot(saccadeAmplitudeList, saccadeVelocityList, 'o')
plt.ylabel('Saccade velocity [in degrees/s]')
plt.xlabel('Saccade amplitude [in degrees]')


Text(0.5, 0, 'Saccade amplitude [in degrees]')

In [42]:
saccadeDurationList

[0.011111111111111112,
 0.044444444444444446,
 0.011111111111111112,
 0.011111111111111112,
 0.044444444444444446,
 0.05555555555555555,
 0.011111111111111112,
 0.011111111111111112,
 0.011111111111111112,
 0.022222222222222223,
 0.044444444444444446,
 0.03333333333333333,
 0.03333333333333333,
 0.05555555555555555,
 0.03333333333333333,
 0.011111111111111112,
 0.011111111111111112,
 0.03333333333333333,
 0.07777777777777778,
 0.044444444444444446,
 0.03333333333333333,
 0.06666666666666667,
 0.011111111111111112,
 0.022222222222222223,
 0.022222222222222223,
 0.011111111111111112,
 0.011111111111111112,
 0.03333333333333333,
 0.022222222222222223,
 0.011111111111111112,
 0.022222222222222223,
 0.03333333333333333,
 0.011111111111111112,
 0.011111111111111112,
 0.011111111111111112,
 0.022222222222222223,
 0.011111111111111112,
 0.03333333333333333,
 0.03333333333333333,
 0.022222222222222223,
 0.022222222222222223,
 0.022222222222222223,
 0.07777777777777778,
 0.011111111111111112,
 0